In [1]:
# ライブラリをインポート
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optimizers
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence
from utils import Vocab
from utils.torch import DataLoader

np.random.seed(123)
torch.manual_seed(123)

In [2]:
class EarlyStopping:
    '''
    早期終了 (early stopping)
    '''
    def __init__(self, patience=0, verbose=0):
        self._step = 0
        self._loss = float('inf')
        self.patience = patience
        self.verbose = verbose

    def __call__(self, loss):
        if self._loss < loss:
            self._step += 1
            if self._step > self.patience:
                if self.verbose:
                    print('early stopping')
                return True
        else:
            self._step = 0
            self._loss = loss

        return False
    
def sort(x, t):
    lens = [len(i) for i in x]
    indices = sorted(range(len(lens)), key=lambda i: -lens[i])
    x = [x[i] for i in indices]
    t = [t[i] for i in indices]
    return (x, t)

In [7]:
# データの準備
data_dir = os.getcwd() + "\\data\\small_parallel_enja-master"
en_train_path = os.path.join(data_dir, "train.en")
en_val_path = os.path.join(data_dir, "dev.en")
en_test_path = os.path.join(data_dir, "test.en")

ja_train_path = os.path.join(data_dir, "train.ja")
ja_val_path = os.path.join(data_dir, "dev.ja")
ja_test_path = os.path.join(data_dir, "test.ja")

en_vocab = Vocab()
ja_vocab = Vocab()

en_vocab.fit(en_train_path, encoding="utf-8")
ja_vocab.fit(ja_train_path, encoding="utf-8")

x_train = en_vocab.transform(en_train_path, encoding="utf-8")
x_val = en_vocab.transform(en_val_path, encoding="utf-8")
x_test = en_vocab.transform(en_test_path, encoding="utf-8")

t_train = ja_vocab.transform(ja_train_path, bos=True, eos=True, encoding="utf-8")
t_val = ja_vocab.transform(ja_val_path, bos=True, eos=True, encoding="utf-8")
t_test = ja_vocab.transform(ja_test_path, bos=True, eos=True, encoding="utf-8")

(x_train, t_train) = sort(x_train, t_train)
(x_val, t_val) = sort(x_val, t_val)
(x_test, t_test) = sort(x_test, t_test)

In [8]:
# データローダーの作成
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_dataloader = DataLoader((x_train, t_train), batch_first=True, device=device)
val_dataloader = DataLoader((x_val, t_val), batch_first=True, device=device)
test_dataloader = DataLoader((x_test, t_test), batch_first=True, device=device)

In [13]:
# データの取得
depth_x = len(en_vocab.i2w)
depth_t = len(ja_vocab.i2w)
t_list = []
x_list = []
for (x, t) in train_dataloader:
    x_list.append(x)
    t_list.append(t)

In [31]:
h = 4
d_model = 128
embedding = nn.Embedding(depth_x, d_model, padding_idx=0)

In [32]:
i = 0
source = x_list[i]
x = embedding(source)

In [46]:
x.shape

torch.Size([100, 16, 128])

In [45]:
d_k = d_model // h
W_q = nn.Parameter(torch.Tensor(h, d_model, d_k))

In [39]:
(x.unsqueeze(0).repeat(h, 1, 1, 1))[2, :, :, :]

tensor([[[-0.8554,  0.9074,  0.1261,  ..., -0.1127,  1.0324, -0.6538],
         [-1.1377, -0.8992,  0.1839,  ..., -2.1439,  0.7091,  0.5749],
         [-0.6035,  0.0469, -0.3464,  ...,  0.0236,  1.3681, -0.4501],
         ...,
         [ 0.2403, -0.7947, -0.2931,  ..., -1.2467,  1.1174,  0.3578],
         [ 0.6558, -1.3517,  1.0385,  ...,  0.2543,  0.3051,  0.6280],
         [ 0.3025, -0.1583, -1.1843,  ..., -0.7565,  0.6520,  0.9331]],

        [[-0.9348,  0.0057,  2.4230,  ...,  0.2821, -1.3486,  0.1209],
         [ 0.4774, -0.9994,  0.3008,  ...,  0.1884, -0.0973,  0.4717],
         [ 1.2088, -0.4727,  0.2864,  ...,  0.0691, -1.5390, -0.4413],
         ...,
         [-0.1820,  0.8863,  0.9595,  ..., -0.7298, -1.0500, -0.1551],
         [-0.7451,  1.7010, -2.0380,  ...,  0.0298,  0.0683, -1.1922],
         [ 0.3025, -0.1583, -1.1843,  ..., -0.7565,  0.6520,  0.9331]],

        [[-0.3765,  0.5637,  0.6893,  ...,  0.8063, -1.2274, -0.4222],
         [-0.2369,  0.7803, -2.0637,  ..., -1

In [36]:
x.shape

torch.Size([100, 16, 128])

In [28]:
torch.randn(1, 2, 3, 4).squeeze(1).shape

torch.Size([1, 2, 3, 4])

In [ ]:
# モデルの構築
nn.Embedding()

In [ ]:
# モデルの構築
# Encoder層を定義
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, maxlen=20, device="cpu"):
        super().__init__()
        self.device =device
        self.embedding = nn.Embedding(input_dim, hidden_dim, padding_idx=0)
        self.lstm = nn.LSTM(hidden_dim, hidden_dim)
        
        nn.init.xavier_normal_(self.lstm.weight_ih_l0)
        nn.init.orthogonal_(self.lstm.weight_hh_l0)

    def forward(self, x):
        len_source_sequences = (x.T > 0).sum(dim=-1)
        x = self.embedding(x)
        x = pack_padded_sequence(x, len_source_sequences)
        h, states = self.lstm(x)
        h, _ = pad_packed_sequence(h)
        return h, states
    
# Decoder層を定義
class Decoder(nn.Module):
    def __init__(self, hidden_dim, output_dim, device="cpu"):
        super().__init__()
        self.device = device
        self.embedding = nn.Embedding(output_dim, hidden_dim, padding_idx=0)
        self.lstm = nn.LSTM(hidden_dim, hidden_dim)
        self.attn = Attention(hidden_dim, hidden_dim, device=self.device)
        self.out = nn.Linear(hidden_dim, output_dim)
        
        nn.init.xavier_normal_(self.lstm.weight_ih_l0)
        nn.init.orthogonal_(self.lstm.weight_hh_l0)
        nn.init.xavier_normal_(self.out.weight)

    def forward(self, x, hs, states, source=None):
        x = self.embedding(x)
        ht, states = self.lstm(x, states)
        ht = self.attn(ht, hs, source=source)
        y = self.out(ht)
        return y, states
    
# Attention層を定義
class Attention(nn.Module):
    def __init__(self, output_dim, hidden_dim, device="cpu"):
        super().__init__()
        self.device = device
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim

        self.W_a = nn.Parameter(torch.Tensor(hidden_dim, hidden_dim))
        self.W_c = nn.Parameter(torch.Tensor(hidden_dim + hidden_dim, output_dim))
        self.b = nn.Parameter(torch.zeros(output_dim))

        nn.init.xavier_normal_(self.W_a)
        nn.init.xavier_normal_(self.W_c)

    def forward(self, ht, hs, source=None):
        # スコア関数の計算
        score = torch.einsum("jik, kl->jil", (hs, self.W_a))
        score = torch.einsum("jik, lik->jil", (ht, score))

        # スコア関数を正規化
        score = score - torch.max(score, dim=-1, keepdim=True)[0]
        score = torch.exp(score)
        if source is not None:
            mask_source = source.t().eq(0).unsqueeze(0)   # バンディング部分を求める
            score.data.masked_fill_(mask_source, 0)   #マスク処理
        a = score / torch.sum(score, dim=-1, keepdim=True)

        # 文脈ベクトルの計算
        c = torch.einsum("jik, kil->jil", (a, hs))

        # 出力の計算
        h = torch.cat((c, ht), -1)
        return torch.tanh(torch.einsum("jik, kl->jil", (h, self.W_c)) + self.b)
    
# Encoder Decoderモデルを定義
class EncoderDecoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, maxlen=20, device="cpu"):
        super().__init__()
        self.device = device
        self.encoder = Encoder(input_dim, hidden_dim, device=device)
        self.decoder = Decoder(hidden_dim, output_dim, device=device)

        self.maxlen = maxlen
        self.output_dim = output_dim

    def forward(self, source, target=None, use_teacher_forcing=False):
        batch_size = source.size(1)
        if target is not None:
            len_target_sequences = target.size(0)
        else:
            len_target_sequences = self.maxlen

        hs, states = self.encoder(source)
        y = torch.ones((1, batch_size), dtype=torch.long, device=self.device)
        output = torch.zeros((len_target_sequences, batch_size, self.output_dim), device=self.device)

        for t in range(len_target_sequences):
            out, states = self.decoder(y, hs, states, source=source)
            output[t] = out

            if use_teacher_forcing and target is not None:
                y = target[t].unsqueeze(0)        
            else:
                y = out.max(-1)[1]
        return output

In [ ]:
# モデルの学習
# データの定義
depth_x = len(en_vocab.i2w)
depth_t = len(ja_vocab.i2w)
input_dim = depth_x
hidden_dim = 128
output_dim = depth_t

# アルゴリズムの定義
epochs = 30
model = EncoderDecoder(input_dim, hidden_dim, output_dim, device=device).to(device)
criterion = nn.CrossEntropyLoss(reduction="mean", ignore_index=0)
optimizer = optimizers.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), amsgrad=True)

def compute_loss(label, pred):
    return criterion(pred, label)

def train_step(x, t, teacher_forcing_rate=0.5):
    use_teacher_forcing = (random.random() < teacher_forcing_rate)
    model.train()
    preds = model(x, t, use_teacher_forcing=use_teacher_forcing)
    loss = compute_loss(t.reshape(-1), preds.view(-1, preds.size(-1)))
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss, preds

def val_step(x, t):
    model.eval()
    preds = model(x, t, use_teacher_forcing=False)
    loss = compute_loss(t.reshape(-1), preds.view(-1, preds.size(-1)))
    return loss, preds

def test_step(x):
    model.eval()
    preds = model(x)
    return preds

In [ ]:
# 確率的勾配法でモデルを学習
for epoch in range(epochs):
    print("-" * 20)
    print("epoch: {}".format(epoch+1))

    train_loss = 0.0
    val_loss = 0.0

    for (x, t) in train_dataloader:
        loss, _ = train_step(x, t)
        train_loss += loss.item()

    train_loss /= len(train_dataloader)

    for (x, t) in val_dataloader:
        loss, _ = val_step(x, t)
        val_loss += loss.item()

    val_loss /= len(val_dataloader)
    print('loss: {:.3f}, val_loss: {:.3}'.format(
        train_loss,
        val_loss
    ))

    for idx, (x, t) in enumerate(test_dataloader):
        preds = test_step(x)

        source = x.T.reshape(-1).tolist()
        target = t.T.reshape(-1).tolist()
        out = preds.max(dim=-1)[1].T.reshape(-1).tolist()

        source = ' '.join(en_vocab.decode(source))
        target = ' '.join(ja_vocab.decode(target))
        out = ' '.join(ja_vocab.decode(out))

        print('>', source)
        print('=', target)
        print('<', out)
        print()

        if idx >= 9:
            break